In [1]:
import requests
import json
import pandas as pd

In [2]:
res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states")

In [3]:
states = pd.DataFrame(json.loads(res.text)["states"])

In [4]:
states.to_csv("../data/states.csv", index = False) #sates with id 

In [5]:
df_states = pd.read_csv("../data/states.csv")

In [6]:
df_states.head(5)

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar


In [7]:
df_district_all = None
for state_id in df_states["state_id"].values.tolist():
    state_name = df_states.loc[df_states['state_id'] == state_id, "state_name"].item()
    res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id))
    df_district = pd.DataFrame(json.loads(res.text)["districts"])
    df_district["state_name"] = state_name
    if df_district_all is None:
        df_district_all = df_district
    else:
        df_district_all = pd.concat([df_district_all, df_district], ignore_index=True)

df_district_all = df_district_all.sort_values("district_name")
df_district_all.to_csv("../data/districts.csv", index=False)    #District_id

In [8]:
df_district_all

,district_id,district_name,state_name
598,582,Adilabad,Telangana
326,320,Agar,Madhya Pradesh
324,796,Agatti Island,Lakshadweep
639,622,Agra,Uttar Pradesh
158,154,Ahmedabad,Gujarat
...,...,...,...
307,285,Yadgir,Karnataka
220,197,Yamunanagar,Haryana
492,478,Yanam,Puducherry
411,368,Yavatmal,Maharashtra


In [9]:
df_district = df_district_all.loc[df_district_all["state_name"]== "Gujarat"]
df_district

,district_id,district_name,state_name
158,154,Ahmedabad,Gujarat
159,770,Ahmedabad Corporation,Gujarat
160,174,Amreli,Gujarat
161,179,Anand,Gujarat
162,158,Aravalli,Gujarat
163,159,Banaskantha,Gujarat
164,180,Bharuch,Gujarat
165,175,Bhavnagar,Gujarat
166,771,Bhavnagar Corporation,Gujarat
167,176,Botad,Gujarat


In [25]:
df_district_all = pd.read_csv("../data/districts.csv")
selected_state = "Gujarat"
district_list = df_district_all["district_name"].tolist()
district_list

['Adilabad',
 'Agar',
 'Agatti Island',
 'Agra',
 'Ahmedabad',
 'Ahmedabad Corporation',
 'Ahmednagar',
 'Aizawl East',
 'Aizawl West',
 'Ajmer',
 'Akola',
 'Alappuzha',
 'Aligarh',
 'Alipurduar District',
 'Alirajpur',
 'Almora',
 'Alwar',
 'Ambala',
 'Ambedkar Nagar',
 'Amethi',
 'Amravati',
 'Amreli',
 'Amritsar',
 'Amroha',
 'Anand',
 'Anantapur',
 'Anantnag',
 'Angul',
 'Anjaw',
 'Anuppur',
 'Aranthangi',
 'Araria',
 'Aravalli',
 'Ariyalur',
 'Arwal',
 'Ashoknagar',
 'Attur',
 'Auraiya',
 'Aurangabad',
 'Aurangabad ',
 'Ayodhya',
 'Azamgarh',
 'BBMP',
 'Badaun',
 'Bagalkot',
 'Bageshwar',
 'Baghpat',
 'Bahraich',
 'Baksa',
 'Balaghat',
 'Balangir',
 'Balarampur',
 'Balasore',
 'Ballia',
 'Balod',
 'Baloda bazar',
 'Balrampur',
 'Banaskantha',
 'Banda',
 'Bandipore',
 'Bangalore Rural',
 'Bangalore Urban',
 'Banka',
 'Bankura',
 'Banswara',
 'Barabanki',
 'Baramulla',
 'Baran',
 'Bareilly',
 'Bargarh',
 'Barmer',
 'Barnala',
 'Barpeta',
 'Barwani',
 'Basirhat HD (North 24 Parganas)

## Search by district id 

In [10]:
#Example

district_id = "240" #Ranchi 
date_from = "01-05-2021"

In [11]:
URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}" \
      .format(district_id,date_from)
res = requests.get(URL)

In [18]:
calender_df = pd.DataFrame(json.loads(res.text)["centers"])

In [19]:
calender_df

,center_id,name,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions,vaccine_fees
0,583628,Resaldar Baba Doranda,Jharkhand,Ranchi,Ranchi Urban,834001,23,84,09:00:00,17:00:00,Free,[{'session_id': '3dd65f93-2b54-4fc2-b4ac-5931f...,NaN
1,607446,SEVA SADAN,Jharkhand,Ranchi,Ranchi Urban,834001,0,0,10:00:00,18:00:00,Paid,[{'session_id': '32acb0e5-bf21-4998-84d3-32483...,NaN
2,630703,TAMAR PANCHAYAT,Jharkhand,Ranchi,Tamar,835225,23,85,09:00:00,17:00:00,Free,[{'session_id': '512b0f90-b0a5-489d-b661-ee736...,NaN
3,672725,GURUDWARA PISKA MORE,Jharkhand,Ranchi,Ranchi Urban,834005,23,85,09:00:00,17:00:00,Free,[{'session_id': 'd1592630-b864-4785-ae39-3f74f...,NaN
4,563481,NAMKUM PANCHYAT,Jharkhand,Ranchi,Namkum,834010,0,0,09:00:00,17:00:00,Free,[{'session_id': '1f8956a3-bb1f-4d87-9b5e-31546...,NaN
5,591735,DSP HQ,Jharkhand,Ranchi,Ranchi Urban,834001,23,85,09:00:00,17:00:00,Free,[{'session_id': '3783acfa-2c56-4b37-a853-9ed77...,NaN
6,672730,BOOTI GOVT SCHOOL,Jharkhand,Ranchi,Ranchi Urban,834001,22,84,09:00:00,17:00:00,Free,[{'session_id': '4cfda6de-95dd-4441-8b5e-7ce5c...,NaN
7,630585,CHANHO PANCHYAT,Jharkhand,Ranchi,Chanho,835214,0,0,09:00:00,17:00:00,Free,[{'session_id': '811bb9a1-b2c3-4062-a8d9-9bd7c...,NaN
8,577856,BURMU CHC,Jharkhand,Ranchi,Burmu,835214,23,85,09:00:00,17:00:00,Free,[{'session_id': '2965f676-f7ec-4f1a-bbd7-8f2cd...,NaN
9,577897,ORMANJHI CHC,Jharkhand,Ranchi,Ormanjhi,835219,23,85,09:00:00,17:00:00,Free,[{'session_id': '694a379b-84b9-455b-869c-2140e...,NaN


In [17]:
district_pincode_list = calender_df.loc[calender_df['district_name'] == "Ranchi", "pincode"]
district_pincode_list.tolist()

[834001,
 834001,
 835225,
 834005,
 834010,
 834001,
 834001,
 835214,
 835214,
 835219,
 834004,
 834004,
 834001,
 834005,
 834001,
 834001,
 834002,
 835012,
 834001,
 834002,
 834002,
 834001,
 835217,
 834002,
 834009,
 834001,
 834001,
 835202,
 834003,
 834001,
 834009,
 834001,
 835222,
 834002,
 834001,
 834004,
 834001,
 834006,
 835204,
 834006,
 834001,
 834010,
 835234,
 835222,
 834006,
 835234,
 835214,
 835303,
 835222,
 835225,
 835214,
 835214,
 834001,
 835301,
 834006,
 834001,
 835204,
 835101,
 834009,
 834001]

In [16]:
calender_df = calender_df[['name', 'state_name', 'district_name',
       'pincode', 'fee_type','vaccine_fees']]

In [17]:
calender_df.columns

Index(['name', 'state_name', 'district_name', 'pincode', 'fee_type',
       'vaccine_fees'],
      dtype='object')

In [18]:
calender_df.to_csv("../data/vaccine_deatils_7days.csv", index=False)

In [19]:
calender_df.head(10)

,name,state_name,district_name,pincode,fee_type,vaccine_fees
0,Resaldar Baba Doranda,Jharkhand,Ranchi,834001,Free,NaN
1,LAPUNG COVAXIN,Jharkhand,Ranchi,835234,Free,NaN
2,SEVA SADAN,Jharkhand,Ranchi,834001,Paid,"[{'vaccine': 'COVISHIELD', 'fee': '0'}]"
3,TAMAR PANCHAYAT,Jharkhand,Ranchi,835225,Free,NaN
4,JAGUAR,Jharkhand,Ranchi,835222,Free,NaN
5,GURUDWARA PISKA MORE,Jharkhand,Ranchi,834005,Free,NaN
6,NAMKUM PANCHYAT,Jharkhand,Ranchi,834010,Free,NaN
7,JAGARNATH HOSPITAL,Jharkhand,Ranchi,834001,Paid,"[{'vaccine': 'COVAXIN', 'fee': '250'}]"
8,DSP HQ,Jharkhand,Ranchi,834001,Free,NaN
9,BOOTI GOVT SCHOOL,Jharkhand,Ranchi,834001,Free,NaN


In [35]:
import os

In [36]:
os.path.dirname(os.path.abspath(__file__))


NameError: name '__file__' is not defined